In [1]:
# autoimport
%load_ext autoreload
%autoreload 1

In [2]:
import pandas as pd

from pathlib import Path
from src.gnn_models import *

%aimport src.gnn_models

In [3]:
PROCESSED_DATA_DIR = r"E:\gnn_data\processed_step_data_full_node_features"
dataset = FastSTEPDataset(PROCESSED_DATA_DIR)

Fast dataset loaded:
- Total samples: 62591
- Processed successfully: 62591
- Failed processing: 80


In [4]:
dataset_trim = FastSTEPDataset(PROCESSED_DATA_DIR, trim_start=6)

Fast dataset loaded:
- Total samples: 62591
- Processed successfully: 62591
- Failed processing: 80


In [5]:
dataset_trim[0].x

tensor([[-1.4771, -0.6946,  0.1994,  ...,  0.8061, -0.2022, -1.7305],
        [-0.2763,  0.3516, -2.6291,  ..., -0.7756,  0.0729,  0.5812],
        [ 2.2368,  0.1307,  1.6555,  ...,  0.6421, -1.6990,  0.7435],
        ...,
        [ 0.6480,  0.3485,  2.0575,  ...,  0.4278,  1.4448, -0.1901],
        [ 0.6480,  0.3485,  2.0575,  ...,  0.4278,  1.4448, -0.1901],
        [ 0.6480,  0.3485,  2.0575,  ...,  0.4278,  1.4448, -0.1901]])

In [6]:
dataset[0].x

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.8061, -0.2022, -1.7305],
        [ 0.0000,  0.0000,  0.0000,  ..., -0.7756,  0.0729,  0.5812],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.6421, -1.6990,  0.7435],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.4278,  1.4448, -0.1901],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.4278,  1.4448, -0.1901],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.4278,  1.4448, -0.1901]])

In [14]:
# Count zeros and ones in the dataset_trim, and show percentage of each label
from collections import Counter
labels = dataset_trim.get_labels()
label_counts = Counter(dataset_trim.get_labels())
print("Label counts in dataset_trim:")
for label, count in label_counts.items():
    print(f"Label {label}: {count} instances")
# Calculate the percentage of each label
total_count = sum(label_counts.values())
for label, count in label_counts.items():
    percentage = (count / total_count) * 100
    print(f"Label {label}: {percentage:.2f}% of total instances")
class_weights = [len(labels) / count for label, count in
                     label_counts.items()]
print("Class weights for loss function:", class_weights)
# dataset_trim.get_labels()

Label counts in dataset_trim:
Label 1: 39497 instances
Label 0: 23094 instances
Label 1: 63.10% of total instances
Label 0: 36.90% of total instances
Class weights for loss function: None


In [17]:
torch.cuda.empty_cache()
trained_model, history = simple_train_model_v2(
    dataset_trim,
    num_epochs=100,  # Start with just 5 epochs for testing
    batch_size=32,  # Small batch size due to large graphs
    learning_rate=0.002,
    num_graphs_to_use=50000
)

C:\Users\phanm\miniconda3\envs\machine-learning\Lib\site-packages\torch_geometric\deprecation.py:26: UserWarning: 'nn.glob.GlobalAttention' is deprecated, use 'nn.aggr.AttentionalAggregation' instead
  warnings.warn(out)


Limiting training to the first 50000 graphs.
Label 1: 31733 instances
Label 0: 18267 instances
Label 1: 63.47% of total instances
Label 0: 36.53% of total instances
Class weights: tensor([2.7372, 1.5756], device='cuda:0')
Splitting dataset into train and validation sets
Train samples: 40000
Validation samples: 10000
Created GNN model:
- Input features: 64
- Hidden sizes: 128, 64
- Output classes: 2
- Convolution dropout rate: 0.6
- Classifier dropout rate: 0.6

Starting training for 100 epochs...


Epoch 1/100 [Val]: 100%|██████████| 313/313 [01:37<00:00,  3.23it/s, loss=0.6916, acc=36.19%]


Epoch 1/100 - Train Loss: 0.6941, Acc: 54.67% | Val Loss: 0.6931, Acc: 36.19% (Best Val: 36.19%)


Epoch 2/100 [Val]: 100%|██████████| 313/313 [01:37<00:00,  3.21it/s, loss=0.7007, acc=56.99%]


...

Epoch 96/100 - Train Loss: 0.6095, Acc: 70.40% | Val Loss: 0.7771, Acc: 59.80% (Best Val: 64.57%)


Epoch 97/100 [Val]: 100%|██████████| 313/313 [01:32<00:00,  3.39it/s, loss=1.2999, acc=59.59%]


Epoch 97/100 - Train Loss: 0.6087, Acc: 70.50% | Val Loss: 0.7784, Acc: 59.59% (Best Val: 64.57%)


Epoch 98/100 [Train]:  31%|███       | 390/1250 [01:19<02:55,  4.91it/s, loss=0.5857, acc=70.67%]


KeyboardInterrupt: 

In [25]:
labels = dataset.get_labels()
indices = list(range(len(dataset)))
print("Splitting dataset into train and validation sets")
train_indices, val_indices = train_test_split(
    indices,
    test_size=0.2,
    stratify=labels,
    random_state=42
)

print(f"Train samples: {len(train_indices)}")
print(f"Validation samples: {len(val_indices)}")

Splitting dataset into train and validation sets
Train samples: 50072
Validation samples: 12519


In [31]:
from torch.utils.data import Subset

train_dataset = Subset(dataset, train_indices)
val_dataset = Subset(dataset, val_indices)

In [36]:
train_dataset[0].x.shape[1]  # Check number of features

70

In [32]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
# Feature scaling
print("Scaling features...")
raw_features = torch.cat([data.x[:, :6] for data in train_dataset],
                         dim=0).numpy()
scaler = MinMaxScaler()
scaler.fit(raw_features)

for data in train_dataset:
    raw = data.x[:, :6].numpy()
    scaled_raw = torch.tensor(scaler.transform(raw), dtype=torch.float)
    data.x = torch.cat([scaled_raw, data.x[:, 6:]], dim=1)
print("Training dataset features scaled.")
for data in val_dataset:
    raw = data.x[:, :6].numpy()
    scaled_raw = torch.tensor(scaler.transform(raw), dtype=torch.float)
    data.x = torch.cat([scaled_raw, data.x[:, 6:]], dim=1)
print("Validation dataset features scaled.")

In [4]:
new_model = model = SimpleGNN(input_features=70).to(device)

Created GNN model:
- Input features: 70
- Hidden sizes: 128, 64
- Output classes: 2


C:\Users\phanm\miniconda3\envs\machine-learning\Lib\site-packages\torch_geometric\deprecation.py:26: UserWarning: 'nn.glob.GlobalAttention' is deprecated, use 'nn.aggr.AttentionalAggregation' instead
  warnings.warn(out)
